In [1]:
import xmltodict
import os
import pandas as pd
import numpy as np

Скоры на моём тесте и на публичном:
```
* 0.005482 --> 0.0137 (линрег)
* 0.067141 --> 0.0611 (knn)
```
Следующий шаг
* положить в knn не по одному слову из синсета, а все (хотя бы - разделённые через запятую), чтобы обогатить пространство соседей. 
* попробовать выкачать другую модель, с бОльшим количеством слов

# Данные

In [2]:
repo_root = '..'

In [3]:
with open('../data/ruwordnet/synsets.N.xml', 'r', encoding='utf-8') as f:
    synsets_n_raw = xmltodict.parse(f.read(), process_namespaces=True)

In [4]:
with open('../data/ruwordnet/synset_relations.N.xml', 'r', encoding='utf-8') as f:
    rel_n_raw = xmltodict.parse(f.read(), process_namespaces=True)

In [5]:
id2synset = {v['@id']: v for v in synsets_n_raw['synsets']['synset']}

In [6]:
rel_df = pd.DataFrame(rel_n_raw['relations']['relation'])

* гиперним - parent частный случай от child 
* гипоним - child более общий, чем parent
* domain - parent относится к теме child
* POS-synonymy - parent и child из разных частей речи
* part holonym - parent является частью child
* part meronym - parent является надсистемой child
* instance hypernym - parent является одним из элементов, множество которых есть child
* instance hyponym - parent является множеством, включающим в себя child. 
* antonym - противоположности (части речи выглядят покоцанными)

Все свойства строго симметричны (кроме POS-synonymy, которая тоже симметрична, но распределена между разными файлами)

In [7]:
rel_df['parent'] = rel_df['@parent_id'].apply(lambda x: id2synset[x]['@ruthes_name'])
rel_df['child'] = rel_df['@child_id'].apply(lambda x: id2synset.get(x, {}).get('@ruthes_name'))

In [8]:
rel_df['@name'].value_counts()

hyponym              37249
hypernym             37249
domain               27339
POS-synonymy         18137
part holonym          3448
part meronym          3448
instance hyponym      1861
instance hypernym     1861
antonym                442
Name: @name, dtype: int64

In [9]:
rel_df[rel_df['@name']=='hyponym'].sample(10)

,@parent_id,@child_id,@name,parent,child
50369,4068-N,6976-N,hyponym,ГОСУДАРСТВЕННАЯ ФИНАНСОВАЯ ПОДДЕРЖКА,ГОСУДАРСТВЕННОЕ ФИНАНСИРОВАНИЕ
51261,115964-N,115966-N,hyponym,ВТЕРЕТЬ,НАТЕРЕТЬ ПОВЕРХНОСТЬ
85138,106817-N,113648-N,hyponym,"ПРИНУДИТЬ, ЗАСТАВИТЬ",ГАШЕНИЕ ОГНЯ
39061,7670-N,4071-N,hyponym,ОБЩАЯ СОБСТВЕННОСТЬ,ДОЛЕВАЯ СОБСТВЕННОСТЬ
91255,257-N,9221-N,hyponym,АРЕНДНАЯ ПЛАТА,ФРАХТОВЫЙ ПЛАТЕЖ
16364,106846-N,110163-N,hyponym,"ВАРИАНТ, РАЗНОВИДНОСТЬ",ЛЕКАРСТВЕННАЯ ФОРМА
109599,4773-N,146942-N,hyponym,ЖИТЕЛЬ ГОРОДА,ТОКИЙЦЫ
117624,123397-N,125213-N,hyponym,ШАЛУН,"БЕЗОБРАЗНИК, ШКОДНИК"
11113,115387-N,119685-N,hyponym,"РАЗЛИЧНЫЙ, РАЗНООБРАЗНЫЙ",МНОГООБРАЗНЫЙ
62723,207-N,104055-N,hyponym,ПОРТ,АЛИКАНТЕ


In [10]:
rel_n_raw['relations']['relation'][10000]

OrderedDict([('@parent_id', '5417-N'),
             ('@child_id', '2921-N'),
             ('@name', 'domain')])

In [11]:
synsets_n_raw['synsets']['synset'][10]

OrderedDict([('@id', '7793-N'),
             ('@ruthes_name', 'ПОДВЕДОМСТВЕННОСТЬ СУДАМ'),
             ('@definition', ''),
             ('@part_of_speech', 'N'),
             ('sense',
              [OrderedDict([('@id', '7793-N-109004'),
                            ('#text', 'ПОДВЕДОМСТВЕННОСТЬ')]),
               OrderedDict([('@id', '7793-N-208651'),
                            ('#text', 'ПОДВЕДОМСТВЕННОСТЬ СУД')]),
               OrderedDict([('@id', '7793-N-208650'),
                            ('#text', 'ПОДВЕДОМСТВЕННОСТЬ ДЕЛО')]),
               OrderedDict([('@id', '7793-N-215552'),
                            ('#text', 'СУДЕБНЫЙ ПОДВЕДОМСТВЕННОСТЬ')]),
               OrderedDict([('@id', '7793-N-118387'),
                            ('#text', 'СУДЕБНЫЙ ЮРИСДИКЦИЯ')]),
               OrderedDict([('@id', '7793-N-215553'),
                            ('#text', 'ЮРИСДИКЦИЯ СУД')]),
               OrderedDict([('@id', '7793-N-210822'),
                            ('#text', 'НЕП

In [16]:
train_n = pd.read_csv('../data/training_data/training_nouns.tsv', sep='\t', encoding='utf-8')
train_n['parents_list'] = train_n.PARENTS.apply(lambda x: x.split(','))

In [17]:
len(id2synset)

29296

In [18]:
train_n.sample(10)

,SYNSET_ID,TEXT,PARENTS,DEFINITION,parents_list
8070,145491-N,ОРДЕР,"128056-N,2525-N","в античности — архитектурный стиль, определяем...","[128056-N, 2525-N]"
1172,136126-N,БИОГАЗ,"150-N,461-N,133238-N",NaN,"[150-N, 461-N, 133238-N]"
6981,114822-N,СВИНЦОВОСТЬ,"109487-N,106944-N",цветом напоминающий свинец; тёмно-серый,"[109487-N, 106944-N]"
14267,137378-N,ВПИСЫВАНИЕ,"150090-N,111625-N","чертить одну фигуру внутри другой, так, чтобы ...","[150090-N, 111625-N]"
4936,112021-N,БЕНЕФИС,"144510-N,3237-N,112022-N",NaN,"[144510-N, 3237-N, 112022-N]"
11411,153504-N,"СТАФФОРД,АМСТАФФ","108524-N,7678-N",NaN,"[108524-N, 7678-N]"
13276,127489-N,ПЕРЕКАТ,"148413-N,106611-N",NaN,"[148413-N, 106611-N]"
131,112747-N,"ПРОРУБКА,ВЫРУБАНИЕ,ВЫРУБКА,ПРОРУБАНИЕ,ПРОСЕКАН...","112429-N,111853-N","делать отверстие, нарушать целость ударом чего...","[112429-N, 111853-N]"
1249,114955-N,МЕТРО,"3405-N,2525-N",NaN,"[3405-N, 2525-N]"
2808,144882-N,ИЗОЛЕНТА,"133840-N,2409-N",NaN,"[133840-N, 2409-N]"


In [22]:
id2synset['153504-N']

OrderedDict([('@id', '153504-N'),
             ('@ruthes_name', 'СТАФФОРДШИРСКИЙ ТЕРЬЕР'),
             ('@definition', ''),
             ('@part_of_speech', 'N'),
             ('sense',
              [OrderedDict([('@id', '153504-N-749257'),
                            ('#text', 'СТАФФОРДСКИЙ ТЕРЬЕР')]),
               OrderedDict([('@id', '153504-N-749256'),
                            ('#text', 'СТАФФОРДШИРСКИЙ ТЕРЬЕР')]),
               OrderedDict([('@id', '153504-N-749259'),
                            ('#text', 'АМЕРИКАНСКИЙ СТАФФОРДШИРСКИЙ ТЕРЬЕР')]),
               OrderedDict([('@id', '153504-N-128529'),
                            ('#text', 'СТАФФОРД')]),
               OrderedDict([('@id', '153504-N-749258'),
                            ('#text', 'АМСТАФФ')])])])

Все слова из обучающей выборки содержатся и в исходном тезаурусе. Гут. И все родители - тоже. 

In [24]:
len(set(train_n.SYNSET_ID)), len(set(train_n.SYNSET_ID).difference(set(id2synset.keys())))

(12393, 0)

In [25]:
all_parents = {p for pl in train_n.parents_list for p in pl}
len(all_parents), len(all_parents.difference(set(id2synset.keys())))

(8164, 0)

In [26]:
train_n.parents_list.apply(len).describe()

count    16790.000000
mean         2.355152
std          0.555096
min          1.000000
25%          2.000000
50%          2.000000
75%          3.000000
max          5.000000
Name: parents_list, dtype: float64

In [27]:
id2synset['146-N']

OrderedDict([('@id', '146-N'),
             ('@ruthes_name', 'НЕФТЕПРОДУКТ'),
             ('@definition', 'продукт переработки нефти'),
             ('@part_of_speech', 'N'),
             ('sense',
              [OrderedDict([('@id', '146-N-182579'),
                            ('#text', 'ПРОДУКЦИЯ НЕФТЕХИМИЧЕСКИЙ КОМПЛЕКС')]),
               OrderedDict([('@id', '146-N-124154'),
                            ('#text', 'НЕФТЕНАЛИВНОЙ ГРУЗ')]),
               OrderedDict([('@id', '146-N-182580'),
                            ('#text', 'ПРОДУКЦИЯ НЕФТЕПЕРЕРАБОТКА')]),
               OrderedDict([('@id', '146-N-107631'),
                            ('#text', 'НЕФТЕПРОДУКТ')]),
               OrderedDict([('@id', '146-N-109923'),
                            ('#text', 'ПРОДУКТ НЕФТЕПЕРЕРАБОТКА')]),
               OrderedDict([('@id', '146-N-117243'),
                            ('#text', 'НЕФТЯНОЙ ПРОДУКТ')])])])

In [28]:
id2synset['8998-N']

OrderedDict([('@id', '8998-N'),
             ('@ruthes_name', 'УГЛЕВОДОРОД'),
             ('@definition',
              'органическое соединение, молекула которого состоит только из атомов углерода и водорода'),
             ('@part_of_speech', 'N'),
             ('sense',
              OrderedDict([('@id', '8998-N-117378'),
                           ('#text', 'УГЛЕВОДОРОД')]))])

In [29]:
id2synset['133238-N']

OrderedDict([('@id', '133238-N'),
             ('@ruthes_name', 'ОРГАНИЧЕСКОЕ ТОПЛИВО'),
             ('@definition', ''),
             ('@part_of_speech', 'N'),
             ('sense',
              [OrderedDict([('@id', '133238-N-187338'),
                            ('#text', 'ОРГАНИЧЕСКИЙ ТОПЛИВО')]),
               OrderedDict([('@id', '133238-N-691213'),
                            ('#text', 'ОРГАНИЧЕСКИЙ ГОРЮЧЕЕ')])])])

In [30]:
id2synset['56-N']

OrderedDict([('@id', '56-N'),
             ('@ruthes_name', 'ПРОМЫШЛЕННАЯ ПРОДУКЦИЯ'),
             ('@definition', ''),
             ('@part_of_speech', 'N'),
             ('sense',
              [OrderedDict([('@id', '56-N-110034'),
                            ('#text', 'ПРОМЫШЛЕННЫЙ ПРОДУКЦИЯ')]),
               OrderedDict([('@id', '56-N-110039'),
                            ('#text', 'ПРОМЫШЛЕННЫЙ ИЗДЕЛИЕ')]),
               OrderedDict([('@id', '56-N-120875'),
                            ('#text', 'ПРОМЫШЛЕННЫЙ ПРОДУКТ')]),
               OrderedDict([('@id', '56-N-722047'),
                            ('#text', 'ФАБРИЧНЫЙ ПРОДУКЦИЯ')]),
               OrderedDict([('@id', '56-N-103811'),
                            ('#text', 'ПРОДУКЦИЯ ПРОМЫШЛЕННОСТЬ')]),
               OrderedDict([('@id', '56-N-109931'),
                            ('#text',
                             'ПРОДУКЦИЯ ПРОМЫШЛЕННЫЙ ПРЕДПРИЯТИЕ')])])])

Например: МАЗУТ вообще не входит в нашу таксономию, но вот зато он есть в обучающей выборке. 

In [31]:
id2synset['147-N']

OrderedDict([('@id', '147-N'),
             ('@ruthes_name', 'МАЗУТ'),
             ('@definition', ''),
             ('@part_of_speech', 'N'),
             ('sense',
              [OrderedDict([('@id', '147-N-217729'),
                            ('#text', 'ОСТАТОЧНЫЙ НЕФТЕПРОДУКТ')]),
               OrderedDict([('@id', '147-N-217730'),
                            ('#text', 'НЕФТЯНОЙ ОСТАТОК')]),
               OrderedDict([('@id', '147-N-106087'), ('#text', 'МАЗУТ')])])])

In [32]:
rel_df[rel_df.child=='147-N']

,@parent_id,@child_id,@name,parent,child


Возможная стратегия решения:
* Для каждого синсета вычисляем эмбеддинг
* Выучиваем функцию из эмбеддинга гиперонима в эмбеддинг гипонима (можно ещё обратную, для надёжности)
* Для каждого нового слова вычисляем координаты предположительного гипонима. 
* Берём ближайших соседей предположительного гипонима. Это наши кандидаты. 
* Для каждой кандидатной пары применяем скоринговую функцию (от двух эмбеддингов и возможно чего-то ещё), предсказывающую вероятность, что эта пара корректная
* Ранжируем кандидаты-гипонимы этой функцией
* Делаем рекурсивное замыкание этой фигни, на каждом следующем шаге домножая на вероятности-скоры
* Финальный список кандидатов - топ всех результатов рекурсии, отранжированный по произведению всех вероятностей. 

In [33]:
len(id2synset)

29296

# Оценка качества

In [34]:
import sys
import os
def add_sys_path(p):
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('..')

In [35]:
import evaluate

In [36]:
assert evaluate.compute_ap([], [], k=10) == 0

assert evaluate.compute_ap([{'lol'}, {'kek'}], ['kek', 'lol'], k=10) == 1.0
assert evaluate.compute_ap([{'lol'}, {'not kek'}], ['kek'], k=10) == 0.0
assert evaluate.compute_ap([{'lol'}, {'not kek'}, {'cheburek'}], ['not kek'], k=10) == sum([1]) / 3
assert evaluate.compute_ap([{'lol'}, {'not kek'}, {'cheburek'}], ['not kek', 'zhopa'], k=10) == sum([1, 0]) / 3
assert evaluate.compute_ap([{'lol'}, {'not kek'}, {'cheburek'}], ['zhopa', 'not kek'], k=10) == sum([0, 0.5]) / 3

# Бейзлайн

In [37]:
import gensim
# this is araneum 2018 ft model from rusvectores
ft = gensim.models.fasttext.FastTextKeyedVectors.load(
    'C:/Users/ddale/Downloads/NLP/rusvectores/model.model'
)
ft.adjust_vectors()

In [38]:
from nltk import wordpunct_tokenize
from string import punctuation

def tokenize(text):
    return [t for t in wordpunct_tokenize(text.lower()) if not all(c in punctuation for c in t)]

def mean_ft(tokens, n=300):
    vecs = [normalize(ft[w]) for w in tokens]
    if len(vecs) == 0:
        return np.zeros(n)
    return normalize(sum(vecs))

def normalize(v):
    return v / sum(v**2)**0.5

In [39]:
ids = sorted(id2synset.keys())

In [40]:
from tqdm.auto import tqdm, trange

In [41]:
def text2vec(text):
    return mean_ft(tokenize(text))

In [43]:
s = id2synset['147-N']

In [44]:
s

OrderedDict([('@id', '147-N'),
             ('@ruthes_name', 'МАЗУТ'),
             ('@definition', ''),
             ('@part_of_speech', 'N'),
             ('sense',
              [OrderedDict([('@id', '147-N-217729'),
                            ('#text', 'ОСТАТОЧНЫЙ НЕФТЕПРОДУКТ')]),
               OrderedDict([('@id', '147-N-217730'),
                            ('#text', 'НЕФТЯНОЙ ОСТАТОК')]),
               OrderedDict([('@id', '147-N-106087'), ('#text', 'МАЗУТ')])])])

In [45]:
s['@ruthes_name']

'МАЗУТ'

In [46]:
for sense in s['sense']:
    print(sense['#text'])

ОСТАТОЧНЫЙ НЕФТЕПРОДУКТ
НЕФТЯНОЙ ОСТАТОК
МАЗУТ


In [49]:
s['sense']

OrderedDict([('@id', '100007-N-125592'), ('#text', 'ТИХИЙ ОКЕАН')])

In [50]:
{sense['#text'] for sense in s['sense']}

TypeError: string indices must be integers

In [52]:
len(ids_long)

19

In [51]:
id

'100007-N'

In [53]:
s

OrderedDict([('@id', '100007-N'),
             ('@ruthes_name', 'ТИХИЙ ОКЕАН'),
             ('@definition', ''),
             ('@part_of_speech', 'N'),
             ('sense',
              OrderedDict([('@id', '100007-N-125592'),
                           ('#text', 'ТИХИЙ ОКЕАН')]))])

In [59]:
ids_long = []
texts_long = []
for id in ids:
    s = id2synset[id]
    senses = s['sense']
    if not isinstance(senses, list):
        senses = [senses]
    texts = {sense['#text'] for sense in senses}
    texts.add(s['@ruthes_name'])
    if len(texts) > 1:
        texts.add(' ; '.join(sorted(texts)))
    for text in sorted(texts):
        ids_long.append(id)
        texts_long.append(text)
print(len(ids), len(ids_long))

29296 119273


In [60]:
#vecs = np.stack([mean_ft(tokenize(id2synset[idx]['@ruthes_name'])) for idx in tqdm(ids) ])
vecs = np.stack([mean_ft(tokenize(t)) for t in tqdm(texts_long) ])
vecs.shape

(119273, 300)

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
ttrain, ttest = train_test_split(train_n, test_size=0.5, random_state=1)

In [54]:
ttrain.shape

(8395, 5)

In [55]:
ttrain.sample(10)

,SYNSET_ID,TEXT,PARENTS,DEFINITION,parents_list
12235,144195-N,БАНАН,"8534-N,354-N",плод банана,"[8534-N, 354-N]"
13508,138098-N,ЕМКОСТЬ,"110871-N,106841-N",NaN,"[110871-N, 106841-N]"
12052,104516-N,ТАСМАНИЯ,"3465-N,7166-N,107135-N",NaN,"[3465-N, 7166-N, 107135-N]"
2199,105878-N,БАКУРИАНИ,"242-N,123680-N,145516-N",NaN,"[242-N, 123680-N, 145516-N]"
1813,100699-N,ЗЫРЯНОВСК,"242-N,123680-N,145516-N",NaN,"[242-N, 123680-N, 145516-N]"
13056,123882-N,ВСКРЫТИЕ,"121544-N,119282-N",NaN,"[121544-N, 119282-N]"
13480,5970-N,"КОРАБЛЕВОЖДЕНИЕ,СУДОВОЖДЕНИЕ","106614-N,145755-N",NaN,"[106614-N, 145755-N]"
14137,123858-N,КОНТУЗИЯ,"2059-N,113696-N,3406-N","сотрясение, травма, общее поражение организма ...","[2059-N, 113696-N, 3406-N]"
13000,153577-N,ПОЛОСКАНИЕ,"9725-N,7529-N","промыть (рот, горло), полоща водой или каким-л...","[9725-N, 7529-N]"
8512,146369-N,САЛОМАС,"1724-N,552-N",NaN,"[1724-N, 552-N]"


In [56]:
id2idx = {name: idx for idx, name in  enumerate(ids)}

In [57]:
from collections import defaultdict

In [74]:
id2hyponym = defaultdict(set)
id2hypernym = defaultdict(set)

def add_pair(hypo_id, hyper_id):
    if hypo_id in id2hyponym[hyper_id]:
        # the pair is already here
        return 
    if hypo_id in id2hypernym[hyper_id]:
        raise ValueError('{} is already a hypernym of {}, so it cannot become its hyponym'.format(hypo_id, hyper_id))
    for next_hypo in id2hyponym[hypo_id]:
        add_pair(next_hypo, hyper_id)
    for next_hyper in id2hypernym[hyper_id]:
        add_pair(hypo_id, next_hyper)
    id2hyponym[hyper_id].add(hypo_id)
    id2hypernym[hypo_id].add(hyper_id)
        

До обогащения: 28118 ключей, 268637 пар, максимальное число родителей - 63

In [75]:
hypo_df = rel_df[rel_df['@name']=='hyponym']
for r, row in tqdm(hypo_df.iterrows()):
    hypo_id = row['@child_id']
    hyper_id = row['@parent_id']
    add_pair(hypo_id, hyper_id)

In [325]:
print(len(id2hyponym))
print(max(len(c) for c in id2hyponym.values()))
print(max(len(c) for c in id2hypernym.values()))
print(sum(len(c) for c in id2hypernym.values()))

28119
18488
63
268637


In [324]:
import random
key = random.choice(list(id2hypernym.keys()))
print(id2synset[key]['@ruthes_name'])
for v in id2hypernym[key]:
    print(id2synset[v]['@ruthes_name'])

ПОЗДРАВИТЬ
ВЫСКАЗАТЬ МНЕНИЕ
ВЫСКАЗАТЬ
СООБЩИТЬ, УВЕДОМИТЬ
СОГЛАСИТЬСЯ, ДАТЬ СОГЛАСИЕ
ПРОИСХОДЯЩАЯ СУЩНОСТЬ
ПОСТУПОК ЧЕЛОВЕКА
ОДОБРИТЬ
ДЕЙСТВИЕ ЧЕЛОВЕКА
ОТВЕТИТЬ, ПОСТУПИТЬ В ОТВЕТ
ДЕЙСТВИЕ, ЦЕЛЕНАПРАВЛЕННОЕ ДЕЙСТВИЕ


Теперь добавляем объектов из обучающей выборки. Кажется, что после обогащения не особо что-то изменилось. 

In [361]:
ttrain.sample(3)

,SYNSET_ID,TEXT,PARENTS,DEFINITION,parents_list
5038,3191-N,ТЕХЗАДАНИЕ,"5875-N,2428-N",NaN,"[5875-N, 2428-N]"
12704,125133-N,"ВЗДОР,ГЛУПОСТЬ,ДИЧЬ,ЕРУНДА,ЛАБУДА,МУРА,МУТЬ,ТА...","124894-N,107257-N","нечто неважное, бессмысленное, чепуха","[124894-N, 107257-N]"
15350,144328-N,БОЯРЫШНИК,"107842-N,107871-N,144341-N",NaN,"[107842-N, 107871-N, 144341-N]"


In [327]:
for r, row in tqdm(ttrain.iterrows(), total=ttrain.shape[0]):
    hypo_id = row['SYNSET_ID']
    for hyper_id in row.parents_list:
        add_pair(hypo_id, hyper_id)

In [328]:
print(len(id2hyponym))
print(max(len(c) for c in id2hyponym.values()))
print(max(len(c) for c in id2hypernym.values()))
print(sum(len(c) for c in id2hypernym.values()))

28636
18928
63
274902


In [79]:
hypo_indices = []
hyper_indices = []
for key, values in id2hyponym.items():
    for value in values:
        hypo_indices.append(value)
        hyper_indices.append(key)

In [85]:
synX = vecs[[id2idx[ix] for ix in hypo_indices]]
synY = vecs[[id2idx[ix] for ix in hyper_indices]]

In [90]:
synX.var(axis=0).mean()

0.0025339746

In [96]:
(synX-synY).var(axis=0).mean()

0.004187228

In [92]:
from sklearn.linear_model import LinearRegression

In [93]:
linreg = LinearRegression().fit(synX, synY)

In [94]:
pred = linreg.predict(synX)

In [97]:
(synY-pred).var(axis=0).mean()

0.0021273529

Ага! Линейная регрессия немножко помогает :)

In [103]:
pred_norm = (pred.T / ((pred**2).sum(axis=1)**0.5)).T

Удивительно, но после нормализации MSE только возрастает. 

In [104]:
(synY-pred_norm).var(axis=0).mean()

0.0021941925

In [136]:
from sklearn.neighbors import KDTree

In [137]:
tree = KDTree(vecs)

#### пример работы моего алгоритма

In [217]:
row = ttrain.sample(1)

for p in row.parents_list.iloc[0]:
    print(id2synset[p]['@ruthes_name'])
row

ДИПЛОМАТИЧЕСКИЙ РАБОТНИК
ДОЛЖНОСТНОЕ ЛИЦО
РАБОТНИК ЗАГРАНУЧРЕЖДЕНИЯ
ПРЕДСТАВИТЕЛЬ ИНТЕРЕСОВ


,SYNSET_ID,TEXT,PARENTS,DEFINITION,parents_list
13636,127185-N,НУНЦИЙ,"1179-N,3361-N,8688-N,106891-N",NaN,"[1179-N, 3361-N, 8688-N, 106891-N]"


ТЕКСТИЛЬЩИК
РАБОТНИК ЛЕГКОЙ ПРОМЫШЛЕННОСТИ


In [174]:
vec = vecs[id2idx[row.SYNSET_ID.iloc[0]]].reshape(1, -1)

In [175]:
vec_pred = normalize(linreg.predict(vec).ravel())

In [176]:
distances, indices = tree.query(vec_pred.reshape(1, -1), k=10)

In [177]:
for i, d in zip(indices.ravel(), distances.ravel()):
    print(d, ids[i], id2synset[ids[i]]['@ruthes_name'])

0.6634564139419517 150414-N БИОЛОГИЧЕСКАЯ СУЩНОСТЬ
0.6879088440684842 147509-N ЭНЕРГИЯ ЧЕЛОВЕКА
0.6916229001163239 145529-N ФУНКЦИОНАЛЬНАЯ НЕДОСТАТОЧНОСТЬ ОРГАНА
0.6940315004049996 147134-N ФИЗИЧЕСКАЯ СУЩНОСТЬ
0.6955260006298274 142922-N КОНСТИТУЦИОНАЛЬНЫЕ ОСОБЕННОСТИ ОРГАНИЗМА
0.6966659523655019 150555-N ЧЕЛОВЕК ПО СФЕРЕ ДЕЯТЕЛЬНОСТИ
0.6983008243630823 140126-N СОЦИАЛЬНО-ДЕМОГРАФИЧЕСКАЯ ОБЩНОСТЬ
0.7105105770528403 139946-N СОЦИАЛЬНАЯ ОБЩНОСТЬ
0.722326210037546 153782-N ПОСТОЯННАЯ СУЩНОСТЬ
0.7238719449947625 125061-N СУБЪЕКТ СУЖДЕНИЯ


In [204]:
def vec2hypo(vec):
    vec_pred = normalize(linreg.predict(vec.reshape(1, -1)).ravel())
    distances, indices = tree.query(vec_pred.reshape(1, -1), k=10)
    return [ids[i] for i in indices.ravel()]

def get_hypotheses(id):
    vec = vecs[id2idx[id]]
    return vec2hypo(vec)

In [205]:
get_hypotheses('8667-N')

['150414-N',
 '147509-N',
 '145529-N',
 '147134-N',
 '142922-N',
 '150555-N',
 '140126-N',
 '139946-N',
 '153782-N',
 '125061-N']

In [182]:
ttest_hypotheses = [get_hypotheses(id) for id in tqdm(ttest.SYNSET_ID)]

In [185]:
ttest_gt = ttest.parents_list.apply(lambda x: [{c} for c in x]).tolist()

In [187]:
ttest_gt[0]

[{'119261-N'}, {'106775-N'}]

In [188]:
ttest_hypotheses[0]

['147134-N',
 '150414-N',
 '153782-N',
 '136567-N',
 '142922-N',
 '153471-N',
 '146671-N',
 '106569-N',
 '123796-N',
 '125061-N']

In [192]:
aps = pd.Series([evaluate.compute_ap(gt, hyps, k=10) for gt, hyps in zip(ttest_gt, ttest_hypotheses)])

In [193]:
aps.describe()

count    8395.000000
mean        0.005482
std         0.037432
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         0.555556
dtype: float64

In [194]:
(aps>0).mean()

0.03704586063132817

In [195]:
aps[aps>0].describe()

count    311.000000
mean       0.147967
std        0.129571
min        0.022222
25%        0.062500
50%        0.111111
75%        0.166667
max        0.555556
dtype: float64

# Сабмитим бейзлайн

In [198]:
public_test_nouns = pd.read_csv('../data/public_test/nouns_public.tsv', header=None, encoding='utf-8')
public_test_nouns.columns = ['noun']

In [199]:
len(set(public_test_nouns['noun'])), len(set(public_test_nouns['noun']).difference())

762

In [202]:
public_test_nouns.sample(5)

,noun
686,ФЛУКТУАЦИЯ
107,ВОСКРЕСНИК
148,ГУДРОН
192,ЖБАН
633,ТАБАКОВОДСТВО


In [203]:
public_test_embs = np.stack([mean_ft(tokenize(x)) for x in tqdm(public_test_nouns['noun']) ])

In [206]:
public_test_hypos = [vec2hypo(v) for v in tqdm(public_test_embs)]

In [209]:
result_nouns = []
result_hyperonyms = []
result_hyper_names = []
for n, h in zip(public_test_nouns.noun, public_test_hypos):
    for hypo in h:
        result_nouns.append(n)
        result_hyperonyms.append(hypo)
        result_hyper_names.append(id2synset[hypo]['@ruthes_name'])
result_df = pd.DataFrame({'noun': result_nouns, 'result': result_hyperonyms, 'result_text': result_hyper_names})
result_df.sample(10)

,noun,result,result_text
5222,ПУБЛИКАТОР,142922-N,КОНСТИТУЦИОНАЛЬНЫЕ ОСОБЕННОСТИ ОРГАНИЗМА
2571,КИПРЕЙ,125061-N,СУБЪЕКТ СУЖДЕНИЯ
435,БАКАЛАВРИАТ,145529-N,ФУНКЦИОНАЛЬНАЯ НЕДОСТАТОЧНОСТЬ ОРГАНА
5188,ПРОУШИНА,119304-N,ПИТАТЕЛЬНЫЙ (СОДЕРЖАЩИЙ НЕОБХОДИМЫЕ ВЕЩЕСТВА)
2324,КАЙТБОРДИНГ,140126-N,СОЦИАЛЬНО-ДЕМОГРАФИЧЕСКАЯ ОБЩНОСТЬ
4539,ПАТИССОН,143716-N,"ПАРАЗИТИЗМ, ПОТРЕБИТЕЛЬСТВО"
3234,ЛОБСТЕР,150414-N,БИОЛОГИЧЕСКАЯ СУЩНОСТЬ
2272,ИСТОРИОГРАФ,142922-N,КОНСТИТУЦИОНАЛЬНЫЕ ОСОБЕННОСТИ ОРГАНИЗМА
1616,ДЕЛИСТИНГ,141259-N,УЧАСТНИК ВНЕШНЕЭКОНОМИЧЕСКОЙ ДЕЯТЕЛЬНОСТИ
2588,КИСЕТ,132381-N,ПОТРЕБИТЕЛЬСКИЕ ТОВАРЫ ДЛИТЕЛЬНОГО ПОЛЬЗОВАНИЯ


In [216]:
result_df.to_csv('nouns_ft_linreg_v1.tsv', sep='\t', encoding='utf-8', header=None, index=None)

# Второй подход: метод ближайших соседей

А что, если находить "родителей", просто глядя на предков похожих текстов?

In [232]:
row = ttrain.sample(1)

for p in row.parents_list.iloc[0]:
    print(id2synset[p]['@ruthes_name'])
row

КОПЧЕНЫЕ МЯСНЫЕ ПРОДУКТЫ
МЯСНЫЕ ПРОДУКТЫ
КОПЧЕНЫЕ ПРОДУКТЫ


,SYNSET_ID,TEXT,PARENTS,DEFINITION,parents_list
12592,109065-N,БЕКОН,"108355-N,108354-N,143722-N",NaN,"[108355-N, 108354-N, 143722-N]"


In [233]:
vec = vecs[id2idx[row.SYNSET_ID.iloc[0]]]

distances, indices = tree.query(vec.reshape(1, -1), k=10)

for i, d in zip(indices.ravel(), distances.ravel()):
    print(d, ids[i], id2synset[ids[i]]['@ruthes_name'])

0.0 109065-N БЕКОН
0.8712710167339206 150730-N ТМИН (ПРЯНОСТЬ)
0.8841478623142824 108934-N ШАМПИНЬОН
0.8960343825610356 144272-N МЯСНОЕ ФИЛЕ
0.8995815147611657 109097-N ВЕРМИШЕЛЬ
0.9025935634314826 109769-N САРДИНА
0.9049464338759658 143187-N ЧИПСЫ
0.9052300484934338 107975-N АНЧОУС
0.9117641744036451 111660-N ПЕРЕЦ (ПРЯНОСТЬ)
0.9121563760060882 109768-N САРДЕЛЬКА


In [263]:
from collections import Counter

In [378]:
text = public_test_nouns.noun.sample(1).iloc[0]
print(text)
vec = text2vec(text)
distances, indices = tree.query(vec.reshape(1, -1), k=10)

hypotheses = Counter()

for i, d in zip(indices.ravel(), distances.ravel()):
    print(d, ids[i], id2synset[ids[i]]['@ruthes_name'])
    for parent in id2hypernym.get(ids[i], set()):
        hypotheses[parent] += 1
print(len(hypotheses))
for hypo, cnt in hypotheses.most_common(10):
    print(cnt, hypo, id2synset[hypo]['@ruthes_name'])

ДАЛЬТОНИК
0.908786353357731 106371-N ДАЛЛАС
0.93271615402465 142698-N ТОНИК
0.9738889412194028 125980-N ДАЛЬНОМЕР
0.9810747714139905 140495-N САНТА-БАРБАРА
0.9834304232131871 105005-N КОЛОМБО
0.9963476631222952 105193-N СЕНТ-ВИНСЕНТ
0.9964537579113368 140703-N РИМЕЙК ФИЛЬМА
0.996730228507747 104693-N СЕНТ-ЛЮСИЯ
0.9986403244813133 136332-N БРЕЙК-ДАНС
1.002067081091842 127132-N ПЕРЕЛЕТ (ПАДЕНИЕ ДАЛЬШЕ ЦЕЛИ)
67
7 153782-N ПОСТОЯННАЯ СУЩНОСТЬ
5 147134-N ФИЗИЧЕСКАЯ СУЩНОСТЬ
4 149182-N ПУБЛИЧНО-ПРАВОВОЕ ОБРАЗОВАНИЕ
4 149189-N СУБЪЕКТ ПРАВА
4 138167-N СУБЪЕКТ ДЕЯТЕЛЬНОСТИ
3 123680-N МУНИЦИПАЛЬНОЕ ОБРАЗОВАНИЕ
3 590-N НАСЕЛЕННЫЙ ПУНКТ
3 2052-N АДМИНИСТРАТИВНО-ТЕРРИТОРИАЛЬНАЯ ЕДИНИЦА
3 145516-N ГОРОДСКОЕ ПОСЕЛЕНИЕ
3 7166-N ГЕОГРАФИЧЕСКИЙ ОБЪЕКТ


In [283]:
def hypotheses_knn(text, index, k=10, verbose=False):
    vec = text2vec(text)
    distances, indices = index.query(vec.reshape(1, -1), k=k)
    hypotheses = Counter()
    for i, d in zip(indices.ravel(), distances.ravel()):
        if verbose:
            print(d, ids[i], id2synset[ids[i]]['@ruthes_name'])
        for parent in id2hypernym.get(ids[i], set()):
            hypotheses[parent] += 1
    if verbose:
        print(len(hypotheses))
    result = []
    for hypo, cnt in hypotheses.most_common(10):
        if verbose:
            print(cnt, hypo, id2synset[hypo]['@ruthes_name'])
        result.append(hypo)
    return result

In [288]:
hypotheses_knn('тетерев', index=tree, verbose=True)

0.0 108058-N ТЕТЕРЕВ
0.7437579398387988 8956-N ГЛУХАРЬ
0.7674193551437258 143552-N СЕМЕЙСТВО ТЕТЕРЕВИНЫЕ
0.7996734758427403 8911-N КУРОПАТКА
0.8146013732630116 107767-N ЖУРАВЛЬ (ПТИЦА)
0.8384177525280063 107811-N ВОРОНА (ПТИЦА)
0.8384968496822458 108189-N ОВСЯНКА (ПТИЦА)
0.8472328710193752 102893-N ТЕТЕРЕВ (ПРИТ. ДНЕПРА)
0.8545029887220364 134699-N РЯБЧИК
0.8639614736208439 107927-N КУКУШКА (ПТИЦА)
21
10 153782-N ПОСТОЯННАЯ СУЩНОСТЬ
10 147134-N ФИЗИЧЕСКАЯ СУЩНОСТЬ
9 963-N ПТИЦА
9 150414-N БИОЛОГИЧЕСКАЯ СУЩНОСТЬ
9 106579-N ЖИВОЙ ОРГАНИЗМ
9 144174-N ЖИВОТНОЕ
8 4091-N ДИКОЕ ЖИВОТНОЕ
8 5681-N ДИКАЯ ПТИЦА
5 141894-N ПЕРНАТАЯ ДИЧЬ
5 145929-N ОХОТНИЧЬЕ-ПРОМЫСЛОВЫЕ ЖИВОТНЫЕ


['153782-N',
 '147134-N',
 '963-N',
 '150414-N',
 '106579-N',
 '144174-N',
 '4091-N',
 '5681-N',
 '141894-N',
 '145929-N']

In [289]:
ttest.sample(3)

,SYNSET_ID,TEXT,PARENTS,DEFINITION,parents_list
4203,131140-N,"ЗАГНОЕНИЕ,ГНОЕНИЕ","120550-N,125011-N,152470-N",NaN,"[120550-N, 125011-N, 152470-N]"
13749,138126-N,УДАРОПРОЧНОСТЬ,"127016-N,118598-N,119010-N",NaN,"[127016-N, 118598-N, 119010-N]"
15441,136755-N,ЭМБОЛИЯ,"136754-N,6129-N,119562-N",NaN,"[136754-N, 6129-N, 119562-N]"


In [296]:
ttest_hypotheses = [hypotheses_knn(txt, index=tree) for txt in tqdm(ttest.TEXT)]

In [298]:
aps = pd.Series([evaluate.compute_ap(gt, hyps, k=10) for gt, hyps in zip(ttest_gt, ttest_hypotheses)])
print((aps>0).mean())
print(aps[aps>0].mean())
aps.describe()

0.3970220369267421
0.16911255808120496


count    8395.000000
mean        0.067141
std         0.121018
min         0.000000
25%         0.000000
50%         0.000000
75%         0.083333
max         1.000000
dtype: float64

In [293]:
public_test_hypos = [hypotheses_knn(t, index=tree) for t in tqdm(public_test_nouns.noun)]

In [362]:
pd.Series(public_test_hypos).apply(len).describe()

count    762.000000
mean       9.994751
std        0.144905
min        6.000000
25%       10.000000
50%       10.000000
75%       10.000000
max       10.000000
dtype: float64

In [294]:
result_nouns = []
result_hyperonyms = []
result_hyper_names = []
for n, h in zip(public_test_nouns.noun, public_test_hypos):
    for hypo in h:
        result_nouns.append(n)
        result_hyperonyms.append(hypo)
        result_hyper_names.append(id2synset[hypo]['@ruthes_name'])
result_df = pd.DataFrame({'noun': result_nouns, 'result': result_hyperonyms, 'result_text': result_hyper_names})
result_df.sample(10)

,noun,result,result_text
3454,МЕДИАНА,106451-N,МЕСТО В ПРОСТРАНСТВЕ
6999,ХАРЧО,107778-N,КУШАНЬЕ
3232,ЛОБСТЕР,820-N,ВЕЩЕСТВО
4438,ПАНТОГРАФ,111816-N,СОЗДАТЕЛЬ
2031,ЗАТОВАРИВАНИЕ,106473-N,"ОБУСЛАВЛИВАТЬ, СПОСОБСТВОВАТЬ"
7499,ЭЛЕФАНТИАЗ,2149-N,ЧЕЛОВЕК
4784,ПНЕВМОКОКК,147134-N,ФИЗИЧЕСКАЯ СУЩНОСТЬ
25,АВАЛЬ,106627-N,ГРУППА ЛЮДЕЙ
5241,ПУСТЕЛЬГА,147134-N,ФИЗИЧЕСКАЯ СУЩНОСТЬ
6930,ФРЕКИНГ,106646-N,ОТНОШЕНИЕ МЕЖДУ СУЩНОСТЯМИ


In [295]:
result_df.to_csv('nouns_ft_knn_v2.tsv', sep='\t', encoding='utf-8', header=None, index=None)